# 第10章: 事前学習済み言語モデル（GPT型）

本章では、GPT型（Transformerのデコーダ型）の事前学習済みモデルを利用して、言語生成、評判分析器（ポジネガ分類器）の構築、ファインチューニング、強化学習などに取り組む。

In [1]:
%%capture
!pip install transformers

## 90. 次単語予測

“The movie was full of"に続くトークン（トークン列ではなく一つのトークンであることに注意せよ）として適切なもの上位10個と、その確率（尤度）を求めよ。ただし、言語モデルへのプロンプトがどのようなトークン列に変換されたか、確認せよ。

In [2]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torch.utils.data import Dataset, DataLoader

In [3]:
%%capture
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [4]:
model.eval()

prompt = "The movie was full of"


encoded_input = tokenizer(prompt, return_tensors='pt')
print(f"inputs_ids shape:{encoded_input['input_ids'].shape}\nattention_mask shape:{encoded_input['attention_mask'].shape}\n")
print(encoded_input)
token = tokenizer.decode(encoded_input['input_ids'][0])
print(f"tokens: {token}\n")

with torch.no_grad():
    output = model(**encoded_input)


logits = output.logits
print(logits.shape) #shape: [B, L, V]

last_token_logits = logits[0, -1, :]  #これで一番最後のトークンの次のロジット
print(last_token_logits.shape)

last_token_logits_prob = F.softmax(last_token_logits, dim=0)

topk = torch.topk(last_token_logits_prob, k=10)

for idx, score in zip(topk.indices, topk.values):
    print(f"{tokenizer.decode(idx)}: {score.item():.4f}")

inputs_ids shape:torch.Size([1, 5])
attention_mask shape:torch.Size([1, 5])

{'input_ids': tensor([[ 464, 3807,  373, 1336,  286]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]])}
tokens: The movie was full of

torch.Size([1, 5, 50257])
torch.Size([50257])
 jokes: 0.0219
 great: 0.0186
 laughs: 0.0115
 bad: 0.0109
 surprises: 0.0107
 references: 0.0105
 fun: 0.0100
 humor: 0.0074
 ": 0.0074
 the: 0.0067


## 91. 続きのテキストの予測

“The movie was full of"に続くテキストを複数予測せよ。このとき、デコーディングの方法や温度パラメータ（temperature）を変えながら、予測される複数のテキストの変化を観察せよ。

In [5]:
#greedy
import time
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

prompt = "The movie was full of"

encoded_input = tokenizer(prompt, return_tensors='pt')

with torch.no_grad():
  start_time = time.time()
  output = model.generate(encoded_input['input_ids'], max_length=15)
  end_time = time.time()

print(f"time: {(end_time - start_time):.4f}")
print(output.shape) #shape: [B,L]
print(tokenizer.decode(output[0], skip_special_tokens=False)) #確率が最も高いトークンを１つずつ順番に選んでいく->greedy decoding デフォはこれっぽい


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


time: 1.3865
torch.Size([1, 15])
The movie was full of jokes and jokes about how the movie was a joke


In [6]:
#beam_serch
import time
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

prompt = "The movie was full of"

encoded_input = tokenizer(prompt, return_tensors='pt')

with torch.no_grad():
  start_time = time.time()
  output = model.generate(encoded_input['input_ids'], max_length=15, num_beams=5)
  end_time = time.time()

print(f"time: {(end_time - start_time):.3f}s")
print(output.shape) #shape: [B,L]
print(tokenizer.decode(output[0], skip_special_tokens=False)) #ビームサーチは貪欲法と比較して処理時間がおそい．


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


time: 2.906s
torch.Size([1, 15])
The movie was full of jokes and jokes and jokes and jokes and jokes and


In [7]:
#topk
import time
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

model.eval()

prompt = "The movie was full of"

encoded_input = tokenizer(prompt, return_tensors='pt')

with torch.no_grad():
  start_time = time.time()
  output = model.generate(
      encoded_input['input_ids'],
      max_length=15,
      top_k=50,
      top_p=0.9,
      temperature=2.0)
  end_time = time.time()

print(f"time: {(end_time - start_time):.3f}s")
print(output.shape) #shape: [B,L]
print(tokenizer.decode(output[0], skip_special_tokens=False))

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


time: 1.174s
torch.Size([1, 15])
The movie was full of jokes and jokes about how the movie was a joke


## 92. 予測されたテキストの確率を計算

“The movie was full of"に続くテキストを予測し、生成された各単語の尤度を表示せよ（生成されるテキストが長いと出力が読みにくくなるので、適当な長さで生成を打ち切るとよい）。

In [8]:
import time

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

prompt = "The movie was full of"
encoded_input = tokenizer(prompt, return_tensors='pt')
input_ids = encoded_input['input_ids']


with torch.no_grad():
  start_time = time.time()

  output = model.generate(
      input_ids,
      max_length=15,
      top_k=50,
      top_p=1.0,
      temperature=1.0,
      return_dict_in_generate=True,
      output_scores=True
  )
  end_time = time.time()



generated_ids = output.sequences[0]
print(f"generated_ids shape: {generated_ids.shape}")
generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
print(f"time: {(end_time - start_time):.3f}s")
print(f"\nGenerated Text:\n{generated_text}\n")

print(len(output.scores)) #tuple: max_len - input_len (=generated_len)
print(output.scores[0].shape) #shape: [1, vocab_size]

for score in output.scores:
  prob = F.softmax(score, dim=1)
  vocab_ids = torch.argmax(score, dim=1) #shape: [batch_size, ]
  tokens = tokenizer.decode(vocab_ids)
  print(f"{tokens}: P={prob[0, vocab_ids].item():.3f}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


generated_ids shape: torch.Size([15])
time: 1.059s

Generated Text:
The movie was full of jokes and jokes about how the movie was a joke

10
torch.Size([1, 50257])
 jokes: P=0.022
 and: P=0.289
 jokes: P=0.099
 about: P=0.206
 how: P=0.100
 the: P=0.085
 movie: P=0.036
 was: P=0.296
 a: P=0.068
 joke: P=0.174


## 93. パープレキシティ

適当な文を準備して、事前学習済み言語モデルでパープレキシティを測定せよ。例えば、

+ The movie was full of surprises
+ The movies were full of surprises
+ The movie were full of surprises
+ The movies was full of surprises

の4文に対して、パープレキシティを測定して観察せよ（最後の2つの文は故意に文法的な間違いを入れた）。

In [9]:
import math

model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

sentences = ['The movie was full of surprises',
             'The movies were full of surprises',
             'The movie were full of surprises',
             'The movies was full of surprises']

def cal_ppl(sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs, labels=inputs['input_ids']) #loss = -1/L Sigma log(wt|wt<)
        loss = outputs.loss
        ppl = math.exp(loss.item()) #tensorだったからitem()でfloat型に変換
    return ppl

def my_cal_ppl(sentence):
    inputs = tokenizer(sentence, return_tensors='pt')
    ids = inputs['input_ids'][0]  # shape: [seq_len]

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits  # shape: [1, seq_len, vocab_size]
    log_probs = F.log_softmax(logits, dim=2)  # softmaxしてlogとる

    loss_sum = 0.0
    token_num = len(ids) - 1

    # t番目の出力は t+1 番目のトークンを予測している
    for i in range(1, len(ids)):
        true_token_id = ids[i].item()
        log_prob = log_probs[0, i - 1, true_token_id].item()
        loss_sum += log_prob

    ppl = math.exp(-loss_sum / token_num)
    return ppl

ppl = [cal_ppl(sentence) for sentence in sentences]
print(ppl)

my_ppl = [my_cal_ppl(sentence) for sentence in sentences]
print(my_ppl)






`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


[99.35393766125713, 126.48173421763245, 278.87789150142044, 274.66100919346405]
[99.3538966517192, 126.48176606948401, 278.8779110327618, 274.66096990289424]


## 94. チャットテンプレート

"What do you call a sweet eaten after dinner?"という問いかけに対する応答を生成するため、チャットテンプレートを適用し、言語モデルに与えるべきプロンプトを作成せよ。また、そのプロンプトに対する応答を生成し、表示せよ。

In [10]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

prompt = "User: What do you call a sweet eaten after dinner? AI: "
encoded_input = tokenizer(prompt, return_tensors='pt')
input_ids = encoded_input['input_ids']

with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=10,
        do_sample=True,
        top_k=50,
        top_p=1.0,
        temperature=1.0
    )

response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("=== 応答 ===")
print(response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


=== 応答 ===
User: What do you call a sweet eaten after dinner? AI: 俺が漓れするい


## 95. マルチターンのチャット

問題94で生成された応答に対して、追加で"Please give me the plural form of the word with its spelling in reverse order."と問いかけたときの応答を生成・表示せよ。また、その時に言語モデルに与えるプロンプトを確認せよ。

In [11]:
second_prompt = response + " User: Please give me the plural form of the word with its spelling in reverse order. AI:"
print(second_prompt)

encoded_input = tokenizer(second_prompt, return_tensors='pt')
input_ids = encoded_input['input_ids']
with torch.no_grad():
    outputs = model.generate(
        input_ids,
        max_new_tokens=10,
        do_sample=True,
        top_k=50,
        top_p=1.0,
        temperature=1.0
    )

second_response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print("=== 応答 ===")
print(second_response)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


User: What do you call a sweet eaten after dinner? AI: 俺が漓れするい User: Please give me the plural form of the word with its spelling in reverse order. AI:
=== 応答 ===
User: What do you call a sweet eaten after dinner? AI: 俺が漓れするい User: Please give me the plural form of the word with its spelling in reverse order. AI:暮に観も歡さ


## 96. プロンプトによる感情分析

事前学習済み言語モデルで感情分析を行いたい。テキストを含むプロンプトを事前学習済み言語モデルに与え、（ファインチューニングは行わずに）テキストのポジネガを予測するという戦略で、[SST-2](https://dl.fbaipublicfiles.com/glue/data/SST-2.zip)の開発データにおける正解率を測定せよ。

In [12]:
%%capture
!wget https://dl.fbaipublicfiles.com/glue/data/SST-2.zip
!unzip SST-2.zip

In [13]:
# データの読み込み
train_df = pd.read_csv("SST-2/train.tsv", sep='\t', )
dev_df = pd.read_csv("SST-2/dev.tsv", sep='\t', )

dev_data = list(zip(dev_df['sentence'], dev_df['label'])) #list of tuple


In [14]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [15]:
def classify(text, model, tokenizer):
  prompt = f"Review: {text} Sentiment: "
  inputs = tokenizer(prompt, return_tensors='pt')

  with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits

  next_token_logits = logits[0, -1, :] #次の単語のlogits

  positive_token_id = tokenizer.encode(' positive')[0]
  negative_token_id = tokenizer.encode(' negative')[0]

  positive_logit = next_token_logits[positive_token_id]
  negative_logit = next_token_logits[negative_token_id]

  return 1 if positive_logit > negative_logit else 0

In [16]:
correct = 0

for tuple in tqdm(dev_data):
  text = tuple[0]
  label = tuple[1]
  pred = classify(text, model, tokenizer)
  if pred == label:
    correct += 1


100%|██████████| 872/872 [03:49<00:00,  3.80it/s]


In [17]:
print(correct/ len(dev_data)) #これで本当にいいのか...

0.6605504587155964


In [21]:
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

text = "hoge hoge"
inputs = tokenizer(text, return_tensors='pt')
print(inputs)

with torch.no_grad():
  outputs = model(**inputs, output_hidden_states=True)
  hidden_state = outputs.hidden_states[-1]
  print(hidden_state.shape)


{'input_ids': tensor([[8873,  469, 8169,  469]]), 'attention_mask': tensor([[1, 1, 1, 1]])}
torch.Size([1, 4, 768])


## 97. 埋め込みに基づく感情分析

事前学習済み言語モデルでテキストをベクトルで表現（エンコード）し、そのベクトルにフィードフォワード層を通すことで極性ラベルを予測するモデルを学習せよ。

## 98. ファインチューニング

問題96のプロンプトに対して、正解の感情ラベルをテキストの応答として返すように事前学習済みモデルをファインチューニングせよ。

## 99. 選好チューニング

問題96のプロンプトに対して、正解の感情ラベルを含むテキストを望ましい応答、間違った感情ラベルを含むテキストを望ましくない応答として、事前学習済み言語モデルを選好チューニング (preference tuning) を実施せよ。選好チューニングのアルゴリズムとしては、近傍方策最適化 (PPO: Proximal Policy Optimization) や直接選好最適化 (DPO: Direct Preference Optimization) などが考えられる。
